In [1]:
from langchain.agents import Tool, initialize_agent
from langchain_groq import ChatGroq
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper
from langchain import LLMMathChain
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import serial
import serial.tools.list_ports
import time
import os
from langchain.tools import tool
from langchain_community.utilities.twilio import TwilioAPIWrapper
from twilio.rest import Client

In [2]:
llm =ChatGroq(groq_api_key='',
             model_name="mixtral-8x7b-32768") 

ACCOUNT_SID = ''
AUTH_TOKEN = ''
TWILIO_PHONE_NUMBER = os.getenv('TWILIO_PHONE_NUMBER', 'whatsapp:+14155238886')
client = Client(ACCOUNT_SID, AUTH_TOKEN)

In [3]:
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [4]:
search = DuckDuckGoSearchRun()
wikipedia = WikipediaAPIWrapper()


# Web Search Tool
search_tool = Tool(
    name="Web Search",
    func=search.run,
    description="A useful tool for searching the Internet to find information on world events, years, dates, issues, etc. Worth using for general topics. Use precise questions.",
)

# Wikipedia Tool
wikipedia_tool = Tool(
    name="Wikipedia",
    func=wikipedia.run,
    description="A useful tool for searching the Internet to find information on world events, issues, dates, years, etc. Worth using for general topics. Use precise questions.",
)


from langchain import LLMMathChain

#llm_math_chain = LLMMathChain(llm=llm, verbose=True) <- Deprecated !!!

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
math_tool = Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.",
    )


def send_message(message):
    try:
        client.messages.create(
            from_=TWILIO_PHONE_NUMBER,
            body=message,
            to='whatsapp:+919908293531'
        )
    except Exception as e:
        logging.error(f"Error sending message: {e}")

@tool
def whatsappmessage_tool(command: str) -> str:
    """
    A tool that allows to send you the message in whats app
    :param command: A string of message that is to send it on whatapp 
    :return: A string describing the result.
    """
    return send_message(command)

whatsapp = Tool(
        name="whatsapp",
        func=whatsappmessage_tool,
        # description="Controls the Arduino LEDs. Use '1' for LED1 color is blue, '2' for LED2 color is green , '3' for LED3 color is white, and '0' to turn off all."
        description="Useful for when you need to send WhatsApp message or share any data on WhatsApp"
    
    )


def control_arduino_led(command):
    """
    Control Arduino LEDs by sending commands.
    :param command: A string ('1', '2', '3' to turn on LEDs; '0' to turn off all)
    :return: A string describing the result.
    """
    try:
        # Connect to Arduino
        arduino = serial.Serial(port='/dev/cu.usbserial-2110', baudrate=9600)
        time.sleep(2)  # Wait for connection to stabilize

        # Validate command and send to Arduino
        if command in ['0', '1', '2', '3']:
            arduino.write(command.encode())
            return f"Sent command '{command}' to Arduino."
        else:
            return "Invalid command. Please use '0', '1', '2', or '3'."

    except Exception as e:
        return f"Error: {e}"

   



@tool
def control_led_tool(command: str) -> str:
    """
    A tool that allows controlling an Arduino's LEDs via serial communication.
    :param command: A string ('1', '2', '3' to turn on LEDs; '0' to turn off all)
    :return: A string describing the result.
    """
    return control_arduino_led(command)



controll_led = Tool(
        name="ControlLED",
        func=control_led_tool,
        description="Controls the Arduino LEDs. Use '1' for LED1 color is blue, '2' for LED2 color is green , '3' for LED3 color is white, and '0' to turn off all."
    )

In [5]:
agent = initialize_agent(
    tools = [search_tool,wikipedia_tool,math_tool,controll_led,whatsapp],
    llm=llm,
    agent='chat-conversational-react-description',
    verbose=True,  
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory,
    handle_parsing_errors=True
)

/var/folders/cb/xyvfl9z12j94vsnkym5lznw80000gp/T/ipykernel_41428/1318111913.py:1: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(


In [6]:
query = "now with this info compose an email to the founder pitching them to use zoho crm"


In [7]:
res = agent.run(query)


/var/folders/cb/xyvfl9z12j94vsnkym5lznw80000gp/T/ipykernel_41428/1170842329.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  res = agent.run(query)




> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Subject: Exciting Opportunity with Zoho CRM\n\nDear Founder,\n\nI hope this email finds you well. I am writing to introduce you to Zoho CRM, a powerful and user-friendly customer relationship management tool that can help take your business to the next level. With features such as lead and contact management, sales pipeline tracking, and automation of repetitive tasks, Zoho CRM can help you streamline your sales and marketing processes and improve your overall efficiency. I would be happy to schedule a demo or provide more information. Please let me know if you are interested.\n\nBest regards,\n[Your Name]"
}
```

> Finished chain.


In [8]:
res

'Subject: Exciting Opportunity with Zoho CRM\n\nDear Founder,\n\nI hope this email finds you well. I am writing to introduce you to Zoho CRM, a powerful and user-friendly customer relationship management tool that can help take your business to the next level. With features such as lead and contact management, sales pipeline tracking, and automation of repetitive tasks, Zoho CRM can help you streamline your sales and marketing processes and improve your overall efficiency. I would be happy to schedule a demo or provide more information. Please let me know if you are interested.\n\nBest regards,\n[Your Name]'